# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on the Boston Housing dataset!

## Objectives
You will be able to:
* Determine if it is necessary to perform normalization/standardization for a specific model or set of data
* Use standardization/normalization on features of a dataset
* Identify if it is necessary to perform log transformations on a set of features
* Perform log transformations on different features of a dataset
* Use statsmodels to fit a multiple linear regression model
* Evaluate a linear regression model by using statistical performance metrics pertaining to overall model and specific parameters


## The Boston Housing Data

We pre-processed the Boston Housing data again. This time, however, we did things slightly different:
- We dropped `'ZN'` and `'NOX'` completely 
- We categorized `'RAD'` in 3 bins and `'TAX'` in 4 bins
- We transformed `'RAD'` and `'TAX'` to dummy variables and dropped the first variable to eliminate multicollinearity
- We used min-max-scaling on `'B'`, `'CRIM'`, and `'DIS'` (and log transformed all of them first, except `'B'`)
- We used standardization on `'AGE'`, `'INDUS'`, `'LSTAT'`, and `'PTRATIO'` (and log transformed all of them first, except for `'AGE'`) 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(['NOX', 'ZN'],axis=1)

# First, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0, 6, 24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# First, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix='TAX', drop_first=True)
rad_dummy = pd.get_dummies(bins_rad, prefix='RAD', drop_first=True)
boston_features = boston_features.drop(['RAD','TAX'], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [2]:
age = boston_features['AGE']
b = boston_features['B']
logcrim = np.log(boston_features['CRIM'])
logdis = np.log(boston_features['DIS'])
logindus = np.log(boston_features['INDUS'])
loglstat = np.log(boston_features['LSTAT'])
logptratio = np.log(boston_features['PTRATIO'])

# Min-Max scaling
boston_features['B'] = (b-min(b))/(max(b)-min(b))
boston_features['CRIM'] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features['DIS'] = (logdis-min(logdis))/(max(logdis)-min(logdis))

# Standardization
boston_features['AGE'] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features['INDUS'] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features['LSTAT'] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features['PTRATIO'] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [3]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,"RAD_(6, 24]","TAX_(270, 360]","TAX_(360, 712]"
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,0,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,0,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,0,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,0,0,0


In [5]:
boston_target = pd.DataFrame(data= np.c_[boston['target']], columns= ['MEDV'])

## Run a linear model in statsmodels

In [6]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [14]:
outcome = 'MEDV'
predictors = ['CRIM', 'INDUS', 'CHAS', 'RM', 'AGE', 'DIS', 'PTRATIO', 'B', 'LSTAT']
pred_sum = '+'.join(predictors)
formula = outcome + '~' + pred_sum

In [15]:
boston_df = pd.concat([boston_target, boston_features], axis=1)
boston_df.head()
model = ols(formula=formula, data=boston_df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                     188.1
Date:                Wed, 04 Mar 2020   Prob (F-statistic):          1.11e-153
Time:                        11:29:52   Log-Likelihood:                -1464.6
No. Observations:                 506   AIC:                             2949.
Df Residuals:                     496   BIC:                             2992.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.3852      3.025      1.780      0.076      -0.558      11.328
CRIM          -0.5796      1.589     -0.365      0.715      -3.702       2.543
INDUS         -1.1717      0.351     -3.339      0.001      -1.861      -0.482
CHAS           2.7949      0.800      3.496      0.001       1.224       4.366
RM             2.8835      0.403      7.164      0.000       2.093       3.674
AGE            0.0107      0.351      0.031      0.976      -0.678       0.700
DIS           -9.9161      1.709     -5.804      0.000     -13.273      -6.559
PTRATIO       -1.3444      0.230     -5.856      0.000      -1.795      -0.893
B              3.9471      0.990      3.986      0.000       2.001       5.893
LSTAT         -5.6282      0.357    -15.757      0.000      -6.330      -4.926
==============================================================================
Omnibus:                      106.690   Durbin-Watson:                   1.098
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              431.721
Skew:                           0.890   Prob(JB):                     1.79e-94
Kurtosis:                       7.160   Cond. No.                         106.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
import statsmodels.api as sm
X_int = sm.add_constant(boston_features)
model = sm.OLS(boston_target,X_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Wed, 04 Mar 2020   Prob (F-statistic):          5.08e-153
Time:                        11:42:14   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              8.6442      3.189      2.711      0.007       2.379      14.910
CRIM              -1.9538      2.115     -0.924      0.356      -6.110       2.202
INDUS             -0.8046      0.362     -2.220      0.027      -1.517      -0.093
CHAS               2.5959      0.796      3.260      0.001       1.032       4.160
RM                 2.6466      0.408      6.488      0.000       1.845       3.448
AGE                0.0794      0.352      0.226      0.821      -0.612       0.770
DIS              -10.0962      1.856     -5.439      0.000     -13.743      -6.449
PTRATIO           -1.4867      0.241     -6.160      0.000      -1.961      -1.013
B                  3.8412      0.986      3.897      0.000       1.905       5.778
LSTAT             -5.6288      0.354    -15.912      0.000      -6.324      -4.934
RAD_(6, 24]        1.3380      0.672      1.990      0.047       0.017       2.659
TAX_(270, 360]    -1.2598      0.600     -2.100      0.036      -2.438      -0.081
TAX_(360, 712]    -2.1461      0.704     -3.047      0.002      -3.530      -0.762
==============================================================================
Omnibus:                      106.730   Durbin-Watson:                   1.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              432.101
Skew:                           0.891   Prob(JB):                     1.48e-94
Kurtosis:                       7.162   Cond. No.                         117.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Run the same model in scikit-learn

In [21]:
from sklearn.linear_model import LinearRegression
y = boston_df['MEDV']
linreg = LinearRegression()
linreg.fit(boston_features, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
linreg.coef_

array([ -1.95380233,  -0.80457549,   2.59586776,   2.64657111,
         0.07939727, -10.09618465,  -1.48666599,   3.8412139 ,
        -5.62879369,   1.33796317,  -1.25977612,  -2.14606188])

In [23]:
linreg.intercept_

8.644156137983725

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of African American individuals by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

In [25]:
crim_log = np.log(0.15)
dis_log = np.log(7.6)
indus_log = np.log(6.07)
lstat_log = np.log(10.87)
ptratio_log = np.log(17)

In [26]:
# Min-Max scaling
new_B = (383-min(b))/(max(b)-min(b))
new_CRIM = (crim_log-min(logcrim))/(max(logcrim)-min(logcrim))
new_DIS = (dis_log-min(logdis))/(max(logdis)-min(logdis))

# Standardization
new_AGE = (33.2-np.mean(age))/np.sqrt(np.var(age))
new_INDUS = (indus_log-np.mean(logindus))/np.sqrt(np.var(logindus))
new_LSTAT = (lstat_log-np.mean(loglstat))/np.sqrt(np.var(loglstat))
new_PTRATIO = (ptratio_log-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [40]:
x_new = [8.6442, new_CRIM, new_INDUS, 1, 6.1, new_AGE, new_DIS, new_PTRATIO, new_B, new_LSTAT, 1, 1, 0]

In [41]:
x_array = np.asarray(x_new)

In [36]:
co_ef = np.asarray([-1.95380233,  -0.80457549,   2.59586776,   2.64657111, 0.07939727, -10.09618465,  -1.48666599,   3.8412139 , -5.62879369,   1.33796317,  -1.25977612,  -2.14606188])

In [37]:
x_array * co_ef

array([-6.47745571e-01,  3.69868634e-01,  2.59586776e+00,  1.61440838e+01,
        2.63598936e+00, -8.10868434e+00,  8.90587933e-01,  1.47118492e+03,
       -1.41039606e-01,  1.33796317e+00, -1.25977612e+00, -0.00000000e+00])

In [42]:
model.predict(x_array)

array([1559.72385427])

In [38]:
sum(x_array * co_ef)

1485.002038692865

## Summary
Congratulations! You pre-processed the Boston Housing data using scaling and standardization. You also fitted your first multiple linear regression model on the Boston Housing data using statsmodels and scikit-learn!